In [1]:
import pandas as pd
import numpy as np

# Tabel 1: Profil Mahasiswa
data_profil = {
    'id_mahasiswa': ['M001', 'M002', 'M003', 'M004'],
    'nama': ['Budi', 'Ani', 'Candra', 'Dewi']
}
df_profil = pd.DataFrame(data_profil)

# Tabel 2: Nilai Mahasiswa
data_nilai = {
    'id_mhs': ['M001', 'M002', 'M003', 'M005'], # ID berbeda
    'nilai_uts': [85, 70, 90, 75],
    'nilai_uas': [90, 85, np.nan, 80]
}
df_nilai = pd.DataFrame(data_nilai)

print("--- Profil Mahasiswa ---")
print(df_profil)
print("\n--- Nilai Mahasiswa ---")
print(df_nilai)

--- Profil Mahasiswa ---
  id_mahasiswa    nama
0         M001    Budi
1         M002     Ani
2         M003  Candra
3         M004    Dewi

--- Nilai Mahasiswa ---
  id_mhs  nilai_uts  nilai_uas
0   M001         85       90.0
1   M002         70       85.0
2   M003         90        NaN
3   M005         75       80.0


- Tugas kita adalah menggabungkannya. Karena nama kolom kuncinya berbeda (id_mahasiswa dan id_mhs), kita harus memberitahu Pandas cara mencocokkannya menggunakan left_on dan right_on.

In [2]:
merge_df = df_profil.merge(df_nilai, left_on='id_mahasiswa', right_on='id_mhs', how='inner')
print(merge_df)

  id_mahasiswa    nama id_mhs  nilai_uts  nilai_uas
0         M001    Budi   M001         85       90.0
1         M002     Ani   M002         70       85.0
2         M003  Candra   M003         90        NaN


- but it's not what our boss wants, right? We've lost Dewi!

In [3]:
keep_dewi_df = df_profil.merge(df_nilai, left_on='id_mahasiswa', right_on='id_mhs', how='left')
print(keep_dewi_df)

  id_mahasiswa    nama id_mhs  nilai_uts  nilai_uas
0         M001    Budi   M001       85.0       90.0
1         M002     Ani   M002       70.0       85.0
2         M003  Candra   M003       90.0        NaN
3         M004    Dewi    NaN        NaN        NaN


In [4]:
keep_dewi_df['nilai_uts'] = pd.to_numeric(keep_dewi_df['nilai_uts'], errors='coerce')
# print("Data Mahasiswa setelah mengubah kolom nilai_uts:", keep_dewi_df, sep="\n")
keep_dewi_df['nilai_uas'] = pd.to_numeric(keep_dewi_df['nilai_uas'], errors='coerce')
# print("Data Mahasiswa setelah mengubah kolom nilai_uas:", keep_dewi_df, sep="\n")

print("=" * 50)
# adding new column nilai_akhir
keep_dewi_df['nilai_akhir'] = (keep_dewi_df['nilai_uts'] * 0.4) + (keep_dewi_df['nilai_uas'] * 0.6)

print("Data Mahasiswa setelah menambahkan kolom nilai_akhir:", keep_dewi_df, sep="\n")

Data Mahasiswa setelah menambahkan kolom nilai_akhir:
  id_mahasiswa    nama id_mhs  nilai_uts  nilai_uas  nilai_akhir
0         M001    Budi   M001       85.0       90.0         88.0
1         M002     Ani   M002       70.0       85.0         79.0
2         M003  Candra   M003       90.0        NaN          NaN
3         M004    Dewi    NaN        NaN        NaN          NaN


- we needs fill nan

In [5]:
# Kita ambil DataFrame hasil 'left join' kita
df_laporan = keep_dewi_df.copy() # Selalu aman untuk bekerja di salinan

# 1. Isi NaN di kolom ASAL (seperti cara pertamamu)
df_laporan['nilai_uts'] = df_laporan['nilai_uts'].fillna(0)
df_laporan['nilai_uas'] = df_laporan['nilai_uas'].fillna(0)

# 2. BARU hitung nilai_akhir (sekarang sudah tidak ada NaN)
df_laporan['nilai_akhir'] = (df_laporan['nilai_uts'] * 0.4) + (df_laporan['nilai_uas'] * 0.6)

print("--- Laporan Final (NaN sudah diisi 0) ---")
print(df_laporan)

--- Laporan Final (NaN sudah diisi 0) ---
  id_mahasiswa    nama id_mhs  nilai_uts  nilai_uas  nilai_akhir
0         M001    Budi   M001       85.0       90.0         88.0
1         M002     Ani   M002       70.0       85.0         79.0
2         M003  Candra   M003       90.0        0.0         36.0
3         M004    Dewi    NaN        0.0        0.0          0.0


In [10]:
df_laporan['status_lulus'] = np.where(df_laporan['nilai_akhir'] > 70, 'LULUS', 'TIDAK LULUS')
print("--- Laporan Final dengan Status Lulus ---")
print(df_laporan[['nama', 'status_lulus', 'nilai_akhir']])

--- Laporan Final dengan Status Lulus ---
     nama status_lulus  nilai_akhir
0    Budi        LULUS         88.0
1     Ani        LULUS         79.0
2  Candra  TIDAK LULUS         36.0
3    Dewi  TIDAK LULUS          0.0


- Bagaimana cara kamu menghitung rata-rata nilai_akhir untuk setiap grup di status_lulus?

In [11]:
avg_status = df_laporan.groupby('status_lulus')['nilai_akhir'].mean().reset_index()
print("--- Rata-rata Nilai Akhir per Status Lulus ---")
print(avg_status)

--- Rata-rata Nilai Akhir per Status Lulus ---
  status_lulus  nilai_akhir
0        LULUS         83.5
1  TIDAK LULUS         18.0
